In [ ]:
from pandas import json_normalize
import pandas as pd
import json
import numpy
from sklearn.cluster import MiniBatchKMeans
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
import operator
import random
import matplotlib.image as mpimg

### Partie Analyse

On a précedemment créé les différents labels pour chaque tableau, initialisé les différents utilisateurs. On peut maintenant analyser les donnés et en déduire d'autres informations qui nous serviront à faire les recommandations pour les utilisateurs.

In [ ]:
#ouverture des fichiers précédemment créés
with open("label.json",'r') as jsonTab:
    dataTab = json.load(jsonTab)
dataframeTab = json_normalize(dataTab)

with open("user.json",'r') as jsonUser:
    dataUser = json.load(jsonUser)
dataframeUser = json_normalize(dataUser)

In [ ]:
n = int(input("Vous souhaiter visualiser les données de l'utilisateur n° ? (entre 0 et 99)"))
if n not in range(99):
    print("Nombre invalide, le script va sûrement planter")

On commence par déterminer la couleur dominante de chaque tableaux, qui va nous permettre de réaliser une recommandation précise pour chaque utilisateur

In [ ]:
#détermination des couleurs et des formats préférés de l'utilisateur en fonction des likes
couleursPref = []
listeFormat = []
compteFormat = []
#récupération des couleurs dominantes des tableaux likés par l'utilisateur 0
for tab in dataTab:
    if tab['lien'] in dataUser[n]['likes']:
        couleursPref.append(tab['couleur'])
        format = tab['format']
        if format in listeFormat:
            compteFormat[listeFormat.index(format)]+=1
        else:
            compteFormat.append(1)
            listeFormat.append(format)

formatPref=listeFormat[compteFormat.index(max(compteFormat))]
dataUser[n]['formatPref']=formatPref

On utilise une fonction KMeans car une moyenne de toutes les couleurs résulterait en une couleur moyenne ~=[122,122,122]. On utilise après un histogramme pour classer les couleurs, et obtenir la couleur qui est la plus représentée sur l'image

In [ ]:

#partitionement des couleurs dominantes de tous les tableaux en 3 cluster
couleursPref = MiniBatchKMeans(3).fit(couleursPref)
#création d'un numpyarray
npbins = numpy.arange(0, 4)
#création d'un histogramme qui classe les clusters par odre croissant
histogram = numpy.histogram(couleursPref.labels_, bins=npbins)
imax = numpy.argmax(histogram[0])

R = math.ceil(couleursPref.cluster_centers_[imax][0])
G = math.ceil(couleursPref.cluster_centers_[imax][1])
B = math.ceil(couleursPref.cluster_centers_[imax][2])

dataUser[n]['couleurPref']=[R,G,B]


On détermine également le tag préféré de chaque utilisateur, afin de pouvoir lui recommander un tableau qui lui corresponde au mieux.<br>
On écrit ensuite ces changements dans le fichier JSON.

In [ ]:
#détermination du tag préféré de l'utilsateur
listeTag=[]
compteTag=[]
for tableau in dataUser[n]['tags']:
    tag=dataUser[n]['tags'][tableau]
    if tag in listeTag:
        compteTag[listeTag.index(tag)]+=1
    else:
        compteTag.append(1)
        listeTag.append(tag)

tagPref=listeTag[compteTag.index(max(compteTag))]
dataUser[n]['tagPref']=tagPref


jsonUser=open("user.json",'w')
jsonUser.write(json.dumps(dataUser, ensure_ascii=False))
jsonUser.close()

### Partie Visualisation

##### Pour pouvoir visualiser les données, et réaliser des classifications on utilise les dataframes de la bibliothèque pandas.

On commence par créer les différents dataframes

In [ ]:
with open("label.json",'r') as jsonTab:
    dataTab = json.load(jsonTab)
dataframeTab = json_normalize(dataTab)

with open("user.json",'r') as jsonUser:
    dataUser = json.load(jsonUser)
dataframeUser = json_normalize(dataUser)

arrayAuteur = []
arrayFormat = []
arrayLikes = []
arrayCouleur = []
for data in dataTab :
    arrayAuteur.append([data['auteur'],data['lien']])
    arrayFormat.append([data['format'],data['lien']])
    arrayLikes.append([data['likes'],data['lien']])
    arrayCouleur.append(data['couleur'])
dataframeAuteur = pd.DataFrame(arrayAuteur, columns=['auteur','lien'])
dataframeFormat = pd.DataFrame(arrayFormat, columns=['format','lien'])
dataframeLikes = pd.DataFrame(arrayLikes, columns=['likes','lien'])
couleursDom = pd.DataFrame(arrayCouleur, columns=['R','G','B'])

On réalise les différents groupby afin d'obtenir les différentes données à utiliser : nombre de tableaux par auteur, par format et par likes

In [ ]:
grouped=dataframeAuteur.groupby("auteur").count()
grouped = grouped.rename(
    columns={'lien':'count'}).reset_index()

grouped2 = dataframeFormat.groupby("format").count()
grouped2 = grouped2.rename(
    columns={'lien':'count'}).reset_index()

grouped3 = dataframeLikes.groupby("likes").count()
grouped3 = grouped3.rename(
    columns={'lien':'count'}).reset_index()

On représente ensuite les couleurs les plus représentées dans nos tableaux

In [ ]:
nbcluster = 5
clusters = MiniBatchKMeans(nbcluster).fit(couleursDom)
#création d'un numpyarray
npbins = numpy.arange(0, nbcluster + 1)
#création d'un histogramme qui classe les clusters par odre croissant
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)

histogram[0][::-1].sort() #permet d'inverser le tri (ordre décroissant)

colors = []
for i in range(nbcluster):
    colors.append('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))


On affiche maintenant les différents éléments : un 'pie chart' pour représenter les couleurs, et des diagrammes batons pour les informations sur les labels

In [ ]:
fig1, ax1 = plt.subplots()
ax1.pie(histogram[0],labels=labels, autopct='%1.1f%%', startangle=90, colors=colors) #autopct permet d'avoir des pourcentages
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

grouped.plot(x=0, kind='bar', title="Nombre de tableaux par auteur")
grouped2.plot(x=0, kind='bar', title="Nombre de tableaux par format")
grouped3.plot(x=0, kind='bar', title="Nombre de tableaux par likes")
plt.show()

### Partie Recommandation

On réalise les recommandations pour les utilisateurs. On a choisi de faire les recommandations pour l'utilisateur 0, mais on pourrait le réaliser pour n'importe quel utilisateur de notre base de doonnées

On crée une fonction comparaison couleur, que nous utiliserons pour chercher si deux couleurs sont proches ou non

In [ ]:
#Fonction de comparaison de couleurs
#Entrées : deux codes RGB de deux couleurs
#Sorties : un indice de proximité des couleurs
def ComparaisonCouleur(couleur1, couleur2):
    dif = []
    for i in range (3):
        dif.append(couleur1[i] - couleur2[i])
    #mise au carré pour être sûr d'avoir des valeurs de diférence positive 
    return dif[0]**2+dif[1]**2+dif[2]**2

On crée ensuite les dataframes que nous donnerons à notre algorithme de classification. On crée donc deux dataframes : celui avec les tableaux (on mettra les différents paramètres de comparaison : auteur, format et tag le plus utilisé). Le dataframe result sera composé de 1 (le tableau a été liké) et de -1 (le tableau a été vu mais non liké)

In [ ]:
data=[]
result=[]
auteur_vu=[]
dataTabCustom = dataTab #on crée une copie de notre liste de tableaux, qui représentera les tableaux "non vus" par l'utilisateur
for lien in dataUser[n]["likes"]: #on construit un dataframe qui va servir à l'algo de classification : on commence par les tableaux likés par l'utilisateur (result = 1)
    for tableau in dataTab:
        if lien == tableau["lien"]:
            tag = max(tableau["tags"].items(), key=operator.itemgetter(1))[0] #l'algo ne prend pas de liste de tags en paramètre: on choisit donc le tag le plus utilisé sur le tableau
            data.append([tableau["auteur"],tableau["format"],tag])
            result.append(1)
            auteur_vu.append(tableau["auteur"])
            dataTabCustom.remove(tableau)
            

for lien in dataUser[n]["unlikes"]:#on fait la même chose avec les tableaux vu et non likés (result = -1)
    for tableau in dataTab:
        if lien == tableau["lien"]:
            tag = max(tableau["tags"].items(), key=operator.itemgetter(1))[0]
            data.append([tableau["auteur"],tableau["format"],tag])
            result.append(-1)
            auteur_vu.append(tableau["auteur"])
            dataTabCustom.remove(tableau)

On lance maintenant notre algorithme de classification : on utilise l'algorithme DecisionTreeClasifier, qui nous permetttra de réaliser des prédictions (l'utilisateur va aimer ou non le tableau).
On commence par encoder les labels pour que l'algorithme puisse les comprendre, puis on fourni les dataframes à l'algorithme

In [ ]:
dataframe = pd.DataFrame(data, columns=['auteur', 'format', 'tags'])#construction des dataframes
resultframe = pd.DataFrame(result, columns=['like'])

le1 = LabelEncoder() #encodage des labels
dataframe['auteur'] = le1.fit_transform(dataframe['auteur'])

le2 = LabelEncoder()
dataframe['format'] = le2.fit_transform(dataframe['format'])

le3 = LabelEncoder()
dataframe['tags'] = le3.fit_transform(dataframe['tags'])

le4 = LabelEncoder()
resultframe['like'] = le4.fit_transform(resultframe['like'])

#On utilise les classifieurs "arbres"
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe)

On réalise maintenant des recommandations jusqu'à avoir 10 recommandations à faire à l'utilisateur. On a supprimé les warnings généré par l'algorithme grâce aux deux premièress lignes.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

recommandation = [] #on initialise une liste de tableaux recommandés
non_recommandation=[] #on initialise une liste de tableaux non-recommandés
while len(recommandation)<10 and len(dataTabCustom)>0:
    tab = random.choice(dataTabCustom)
    if tab["auteur"] not in auteur_vu: #l'algo de prédiction plante si on lui présente un tableau dont l'auteur est n'a pas été vu par l'utilisateur.
        continue
    prediction = dtc.predict([ #on réalise une prédiction pour savoir si le tableau plaira à l'utilisateur
            [le1.transform([tab["auteur"]])[0], le2.transform([tab["format"]])[0],
            le3.transform([max(tableau["tags"].items(), key=operator.itemgetter(1))[0]])[0]]])
    if prediction == 1: 
        recommandation.append(tab)
    else:
        non_recommandation.append(tab)
    dataTabCustom.remove(tab)

On rajoute maintenant le paramètre de la couleur, afin de pouvoir proposer à l'utilisateur le tableau dont la couleur dominante est la plus proche de sa couleur préférée.

In [ ]:
for tab in recommandation: #on ajoute une comparaison par couleurs des tableaux, afin d'avoir la recommandation la plus précise possible
    tab["indice_couleur"]=ComparaisonCouleur(dataUser[n]["couleurPref"],tab["couleur"])

dataframe=pd.DataFrame(recommandation)
dataframe.drop(columns=["auteur",'largeur','hauteur','format','tags','unlikes','couleur'],inplace=True)
dataframe.sort_values(by="indice_couleur",inplace=True)
if dataframe.iloc[0]['indice_couleur']>200 :
    dataframe.sort_values(by="likes",inplace=True)

#### Affichage des recommandations

On affiche les images likés de l'utilisateur, puis les recommandations que nous pouvons faire grâce aux algorithmes

In [ ]:
print('Images likés')
for i in range(len(dataUser[n]["likes"])):
    img = mpimg.imread(dataUser[n]["likes"][i])
    imgplot=plt.imshow(img)
    plt.show()

In [ ]:
print('Notre recommandation : ')
img = mpimg.imread(dataframe.iloc[0]['lien'])
imgplot = plt.imshow(img)
plt.show()



In [ ]:
print("Les tableaux qui pourraient aussi vous plaire : ")
for i in range(9):
    img = mpimg.imread(dataframe.iloc[i+1]['lien'])
    imgplot = plt.imshow(img)
    plt.show()